<a href="https://colab.research.google.com/github/jetafese/btor2mlir/blob/ebpf/utils/ebpf/results/ebpf_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#eBPF Experiment Results


## Setup & Utilities

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy
from matplotlib.backends.backend_pdf import PdfPages

# Do not limit number of rows displayed
pd.set_option('display.max_rows', None)

In [3]:
def clean_csv(in_file, ref=False):
  df = pd.read_csv(in_file)
  # get program name, ignore full path
  df[['path', 'program']] = df['File'].str.split(pat = '/ebpf-samples', expand = True)
  cols = df.columns.tolist()
  cols = cols[-1:] + cols[:-1]
  df = df[cols]
  df = df.drop(columns=['path', 'File'])
  # distinguish between reference results and current results
  if ref:
    df = df.rename(columns = {'Result': 'prevail'})
  else:
    df = df.rename(columns = {'Result': 'result'})

  return df

## Collect CSVs

In [5]:
#include csv for reference, current results
CURRENT = "https://raw.githubusercontent.com/jetafese/btor2mlir/refs/heads/ebpf/utils/ebpf/results/prevail_0918241625.csv"
REFERENCE = "https://raw.githubusercontent.com/jetafese/btor2mlir/refs/heads/ebpf/utils/ebpf/results/ref_prevail_0425241000.csv"

current_results = clean_csv(CURRENT)
ref_results = clean_csv(REFERENCE, ref=True)
# current_results
# ref_results

## Roundtrip checks

### Soundness

In [8]:
# for (x, y) in zip(current_results, ref_results):
df = pd.merge(current_results, ref_results[['program', 'prevail']], how='inner', on='program')
# df[df['prevail'].str.contains("marshal")]
df[df['result'].str.contains("llvm")]
# df['prevail']

,program,result,prevail
31,/bpf_cilium_test/bpf_lxc_jit.o:2/10,error: llvm dialect conversion failed,unmarshaling error at 21: bad instruction op 0x30
32,/bpf_cilium_test/bpf_lxc_jit.o:2/7,error: llvm dialect conversion failed,1
35,/bpf_cilium_test/bpf_lxc_jit.o:1/0xdc06,error: llvm dialect conversion failed,1
89,/cilium/bpf_lxc.o:2/10,error: llvm dialect conversion failed,unmarshaling error at 22: bad instruction op 0x30
111,/cilium/bpf_xdp_dsr_linux.o:2/21,error: llvm dialect conversion failed,0
112,/cilium/bpf_xdp_dsr_linux.o:2/16,error: llvm dialect conversion failed,0
113,/cilium/bpf_xdp_dsr_linux.o:2/18,error: llvm dialect conversion failed,0
114,/cilium/bpf_xdp_dsr_linux.o:2/24,error: llvm dialect conversion failed,0
116,/cilium/bpf_xdp_dsr_linux.o:2/15,error: llvm dialect conversion failed,0
120,/cilium/bpf_xdp_dsr_linux.o:2/10,error: llvm dialect conversion failed,0
